In [1]:
from polyhedral_analysis.polyhedra_recipe import PolyhedraRecipe
from polyhedral_analysis.octahedral_analysis import trans_vector_orthogonality, opposite_vertex_pairs, adjacent_vertex_pairs
from polyhedral_analysis.configuration import Configuration
from polyhedral_analysis.atom import Atom

from pymatgen.core import Structure
from collections import Counter
import vg

from itertools import permutations

from figure_formatting import figure_formatting as ff

ff.set_formatting()

colors = {'blue': '#264653',
          'green': '#2A9D8F',
          'yellow': '#E9C46A',
          'light orange': '#F4A261',
          'dark orange': '#E76F51',
          'pink': '#F3B6A5'}

from scipy.stats import binom
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

from scipy.stats import gaussian_kde

In [2]:
file_paths = [f'../../structure_prediction_dataset/vasp_4x4x4_from_ga/config_{i}/cubic/cp2k_opt/cp2k_opt.cif'
              for i in [1,2,3,4]]

In [3]:
structures = [Structure.from_file(f) for f in file_paths]

1
1
1
1


In [4]:
recipe = PolyhedraRecipe(method='distance cutoff',
                         coordination_cutoff=3.0,
                         central_atoms='Ti',
                         vertex_atoms=['O', 'F'])

In [5]:
configs = [Configuration(structure=s, recipes=[recipe]) for s in structures]

In [6]:
def mad(d: list[float]) -> float:
    d_mean = np.mean(d)
    return np.mean(np.absolute(d - d_mean))

In [7]:
print('\t\tTi–F\t\t\t\t\tTi–O')
print('structure\td_max\td_min\t<d>\t<|d-<d>|>\td_max\td_min\t<d>\t<|d-<d>|>')
output = []
for i, c in enumerate(configs, 1):
    r_Ti_X = {'F': [],
              'O': []}
    for p in c.polyhedra:
        for d, l in p.vertex_distances_and_labels(reference='central_atom'):
            r_Ti_X[l].append(d)
    dF = r_Ti_X['F']
    dO = r_Ti_X['O']

    print(f'{i}\t\t'
          f'{max(dF):.3f}\t{min(dF):.3f}\t{np.mean(dF):.3f}\t{mad(dF):.3f}\t\t'
          f'{max(dO):.3f}\t{min(dO):.3f}\t{np.mean(dO):.3f}\t{mad(dO):.3f}')
    output.append([max(dF), min(dF), np.mean(dF), mad(dF), max(dO), min(dO), np.mean(dO), mad(dO)])
average = np.mean(output, axis=0)
print(f'Average\t\t{average[0]:.3f}\t{average[1]:.3f}\t{average[2]:.3f}\t{average[3]:.3f}\t\t'
      f'{average[4]:.3f}\t{average[5]:.3f}\t{average[6]:.3f}\t{average[7]:.3f}')

print('\t\tx–F\t\t\t\t\tx–O')
print('structure\td_max\td_min\t<d>\t<|d-<d>|>\td_max\td_min\t<d>\t<|d-<d>|>')
output = []
for i, c in enumerate(configs, 1):
    r_centroid_X = {'F': [],
                    'O': []}
    for p in c.polyhedra:
        for d, l in p.vertex_distances_and_labels(reference='central_atom'):
            r_Ti_X[l].append(d)
        for d, l in p.vertex_distances_and_labels(reference='centroid'):
            r_centroid_X[l].append(d)
    dF = r_centroid_X['F']
    dO = r_centroid_X['O']

    print(f'{i}\t\t'
          f'{max(dF):.3f}\t{min(dF):.3f}\t{np.mean(dF):.3f}\t{mad(dF):.3f}\t\t'
          f'{max(dO):.3f}\t{min(dO):.3f}\t{np.mean(dO):.3f}\t{mad(dO):.3f}')
    output.append([max(dF), min(dF), np.mean(dF), mad(dF), max(dO), min(dO), np.mean(dO), mad(dO)])
average = np.mean(output, axis=0)
print(f'Average\t\t{average[0]:.3f}\t{average[1]:.3f}\t{average[2]:.3f}\t{average[3]:.3f}\t\t'
      f'{average[4]:.3f}\t{average[5]:.3f}\t{average[6]:.3f}\t{average[7]:.3f}')


		Ti–F					Ti–O
structure	d_max	d_min	<d>	<|d-<d>|>	d_max	d_min	<d>	<|d-<d>|>
1		2.162	1.879	1.989	0.056		1.909	1.740	1.810	0.033
2		2.171	1.864	1.985	0.057		1.934	1.725	1.812	0.045
3		2.136	1.858	1.988	0.050		1.890	1.750	1.810	0.025
4		2.158	1.878	1.984	0.053		1.917	1.733	1.811	0.034
Average		2.157	1.870	1.987	0.054		1.913	1.737	1.811	0.034
		x–F					x–O
structure	d_max	d_min	<d>	<|d-<d>|>	d_max	d_min	<d>	<|d-<d>|>
1		1.956	1.850	1.912	0.019		1.983	1.896	1.939	0.015
2		1.958	1.841	1.907	0.020		1.995	1.906	1.942	0.016
3		1.970	1.849	1.911	0.020		1.977	1.897	1.939	0.013
4		1.969	1.831	1.907	0.019		1.990	1.906	1.941	0.013
Average		1.963	1.843	1.909	0.020		1.987	1.901	1.940	0.014


In [8]:
def adjacent_ti_ti_angle(atom: Atom) -> float:
    assert(len(atom.in_polyhedra) == 2)
    p1, p2 = atom.in_polyhedra
    p1_vertex_index = p1.vertex_internal_index_from_global_index(atom.index)
    p2_vertex_index = p2.vertex_internal_index_from_global_index(atom.index)
    v1 = p1.vertex_vectors(reference='central_atom')[p1_vertex_index]
    v2 = p2.vertex_vectors(reference='central_atom')[p2_vertex_index]
    angle = vg.angle(v1, v2)
    return angle

def mean_ti_x_ti_angles(configuration: Configuration) -> dict[str: float]:
    ti_o_ti_angles = []
    ti_f_ti_angles = []
    for atom in c.atoms:
        match atom.label:
            case 'O':
                ti_o_ti_angles.append(adjacent_ti_ti_angle(atom))
            case 'F':
                ti_f_ti_angles.append(adjacent_ti_ti_angle(atom))
            case 'Ti':
                pass
    return {'F': np.mean(ti_f_ti_angles),
            'O': np.mean(ti_o_ti_angles)}

In [10]:
print('\t\tX–Ti–X angular distortions\tX-×-X angular distortions\tTi–X–Ti angles\t\t\tTi displacment')
print('structure\t<|90°-XX|>\t<|180°-XX|>\t<|90°-XX|>\t<|180°-XX|>\t<|Ti–F–Ti|>\t<|Ti–O–Ti|>\t<|×--Ti|>')
output = []
for i, c in enumerate(configs, 1):
    cis_angles_centre = []
    trans_angles_centre = []
    cis_angles_centroid = []
    trans_angles_centroid = []
    off_centre_displacements = []
    for p in c.polyhedra:
        ovp = [[v1.index, v2.index] for v1, v2 in opposite_vertex_pairs(p, check=False)]
        trans_angles_centre.extend(p.vertex_angles(ovp, reference='central_atom'))
        trans_angles_centroid.extend(p.vertex_angles(ovp, reference='centroid'))
        avp = [[v1.index, v2.index] for v1, v2 in adjacent_vertex_pairs(p,check=False)] 
        cis_angles_centre.extend(p.vertex_angles(avp, reference='central_atom'))
        cis_angles_centroid.extend(p.vertex_angles(avp, reference='centroid'))
        off_centre_displacements.append(p.off_centre_displacement)
    ti_x_ti_angles = mean_ti_x_ti_angles(c)
    print(f'{i}\t\t'
          f'{np.mean(np.absolute(90-np.array(cis_angles_centre))):.2f}\t\t'
          f'{np.mean(np.absolute(180-np.array(trans_angles_centre))):.2f}\t\t'
          f'{np.mean(np.absolute(90-np.array(cis_angles_centroid))):.2f}\t\t'
          f'{np.mean(np.absolute(180-np.array(trans_angles_centroid))):.2f}\t\t'
          f'{ti_x_ti_angles["F"]:.2f}\t\t{ti_x_ti_angles["O"]:.2f}\t\t'
          f'{np.mean(off_centre_displacements):.2f}')
    output.append([np.mean(np.absolute(90-np.array(cis_angles_centre))),
                   np.mean(np.absolute(180-np.array(trans_angles_centre))),
                   np.mean(np.absolute(90-np.array(cis_angles_centroid))),
                   np.mean(np.absolute(180-np.array(trans_angles_centroid))),
                   ti_x_ti_angles["F"], ti_x_ti_angles["O"],
                   np.mean(off_centre_displacements)])
average = np.mean(output, axis=0)
print(f'Average\t\t{average[0]:.2f}\t\t{average[1]:.2f}\t\t{average[2]:.2f}\t\t{average[3]:.3f}\t\t'
      f'{average[4]:.2f}\t\t{average[5]:.2f}\t\t{average[6]:.2f}')

		X–Ti–X angular distortions	X-×-X angular distortions	Ti–X–Ti angles			Ti displacment
structure	<|90°-XX|>	<|180°-XX|>	<|90°-XX|>	<|180°-XX|>	<|Ti–F–Ti|>	<|Ti–O–Ti|>	<|×--Ti|>
1		4.75		10.17		0.90		1.44		155.24		157.94		0.20
2		5.00		10.33		0.81		1.47		170.10		172.53		0.21
3		4.68		10.15		0.80		1.35		154.77		157.34		0.20
4		4.78		10.25		0.86		1.64		170.45		172.54		0.20
Average		4.80		10.23		0.85		1.471		162.64		165.09		0.20


In [49]:
output = []

print('\t\tF–F\t\t\t\t\tF–O\t\t\t\t\tO–O')
print('structure\td_max\td_min\t<d>\t<|d-<d>|>\td_max\td_min\t<d>\t<|d-<d>|>\td_max\td_min\t<d>\t<|d-<d>|>')
for i, c in enumerate(configs, 1):
    d_XX = {'F': {'F': [],
                  'O': []},
            'O': {'O': []}}
    for p in c.polyhedra:
        for v1, v2 in adjacent_vertex_pairs(p, check=False):
            l1, l2 = sorted([v1.label, v2.label])
            d_XX[l1][l2].append(v1.distance(v2))
    print(f'{i}\t\t',
          f'{max(d_XX["F"]["F"]):.2f}\t',
          f'{min(d_XX["F"]["F"]):.2f}\t',
          f'{np.mean(d_XX["F"]["F"]):.2f}\t',
          f'{mad(d_XX["F"]["F"]):.3f}\t\t',
          f'{max(d_XX["F"]["O"]):.2f}\t',
          f'{min(d_XX["F"]["O"]):.2f}\t',
          f'{np.mean(d_XX["F"]["O"]):.2f}\t',
          f'{mad(d_XX["F"]["O"]):.3f}\t\t',
          f'{max(d_XX["O"]["O"]):.2f}\t',
          f'{min(d_XX["O"]["O"]):.2f}\t',
          f'{np.mean(d_XX["O"]["O"]):.2f}\t',
          f'{mad(d_XX["O"]["O"]):.3f}\t\t')
    output.append([max(d_XX["F"]["F"]), min(d_XX["F"]["F"]), np.mean(d_XX["F"]["F"]), mad(d_XX["F"]["F"]),
                   max(d_XX["F"]["O"]), min(d_XX["F"]["O"]), np.mean(d_XX["F"]["O"]), mad(d_XX["F"]["O"]),
                   max(d_XX["O"]["O"]), min(d_XX["O"]["O"]), np.mean(d_XX["O"]["O"]), mad(d_XX["O"]["O"])])
average = np.mean(output, axis=0)
print('Average\t\t',
      f'{average[0]:.2f}\t',
      f'{average[1]:.2f}\t',
      f'{average[2]:.2f}\t',
      f'{average[3]:.3f}\t\t',
      f'{average[4]:.2f}\t',
      f'{average[5]:.2f}\t',
      f'{average[6]:.2f}\t',      
      f'{average[7]:.3f}\t\t',
      f'{average[8]:.2f}\t',
      f'{average[9]:.2f}\t',
      f'{average[10]:.2f}\t',
      f'{average[11]:.3f}')

		F–F					F–O					O–O
structure	d_max	d_min	<d>	<|d-<d>|>	d_max	d_min	<d>	<|d-<d>|>	d_max	d_min	<d>	<|d-<d>|>
1		 2.80	 2.60	 2.69	 0.028		 2.82	 2.64	 2.74	 0.023		 2.80	 2.69	 2.74	 0.016		
2		 2.76	 2.58	 2.68	 0.023		 2.82	 2.66	 2.74	 0.022		 2.79	 2.70	 2.74	 0.015		
3		 2.78	 2.59	 2.69	 0.025		 2.80	 2.65	 2.74	 0.023		 2.78	 2.71	 2.74	 0.014		
4		 2.77	 2.56	 2.68	 0.025		 2.83	 2.66	 2.74	 0.023		 2.79	 2.70	 2.74	 0.014		
Average		 2.78	 2.58	 2.68	 0.025		 2.82	 2.66	 2.74	 0.023		 2.79	 2.70	 2.74	 0.015


In [44]:
mad(d_XX['F']['F'])

0.02516566524356334